In [ ]:
import numpy as np 
import pandas as pd
import mne
import glob
import os
from pyedflib import highlevel
from matplotlib import pyplot as plt
from utils.freq_calculator import do_bandpass, prepare_fft
import copy
import pyxdf


from utils.lag_calculator import (
    epoch_data,
    remove_outliers,
    replace_outliers,
    calculate_epochs_lag,
    prepare_comparison_data,
    prepare_idun_data,
    adjust_data_by_mean_lag,
    sync_start_and_equalize_data_length,
    clean_data_from_spikes,
    convert_data_to_array,
    get_device_configuration,
    cut_throughout_data_dual,
    interpolate_signal,
    smooth,
    manual_sync
)
import config

### Upload all data

In [ ]:
#pyedflib
comparison_raw_data

In [ ]:
#mne
mne_comparison_raw_data[:]


In [ ]:
# folder = "Fullscalp"
# subject = "SB"
folder = "/Users/eliasmeier/Documents/IDUN/Data/Eurostars_Cerebra/ORP_inear_adaptation/01_Pre_study"
subject = "S001"
night = "night1"
idun_file_ending = "eeg"


if len(glob.glob(os.path.join(folder, subject, night, "*scoring.edf"))) != 0:
    edf_file_path = glob.glob(os.path.join(folder, subject, night, "*scoring.edf"))[0]
    print(edf_file_path)
    mne_comparison_raw_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    comparison_raw_data = highlevel.read_edf(edf_file_path)
    
    # iterate to initial file to match format of mne edf reader
    # TODO: Does this work? Arrays are of different length
    #comparison_raw_data = []
    #for row in edf_data:
    #    comparison_raw_data.append(np.array(row))

    #create timestamp array 
    samp_rate = comparison_raw_data[1][1]['sample_rate']
    sample_count = len(comparison_raw_data[0][1]) # select channel that has 120Hz sampling freq
    increment = 1/samp_rate # increase in these increments based on prodigy EEG channel sampling rate
    comparison_time_stamps = np.arange(0, sample_count*increment, increment)
    
    file_extention = "edf"
else:
    print("No edf file present in the folder")
if len(glob.glob(os.path.join(folder, subject, night, "*.xdf"))) != 0:
    xdf_file_path = glob.glob(os.path.join(folder, subject, night, "*.xdf"))[0]
    print(xdf_file_path)
    comparison_raw_data, comparison_raw_header = pyxdf.load_xdf(xdf_file_path)
    comparison_time_stamps = comparison_raw_data[0]["time_stamps"]
    file_extention = "xdf"
else:
    print("No xdf file present in the folder")

# csv_file_path = glob.glob(os.path.join(folder, subject, night, "*eeg.csv"))[0]
csv_file_path = glob.glob(
    os.path.join(folder, subject, night, f"*{idun_file_ending}.csv")
)[0]

# upload csv file using numpy
idun_raw_data = np.genfromtxt(csv_file_path, delimiter=",", skip_header=1)
# print the paths

print(csv_file_path)
channel_1_key, channel_2_key, scale_factor, sample_rate = get_device_configuration(
    config
)
# print channel keys
print(channel_1_key, channel_2_key)

### Unpack scalp data

In [ ]:
comparison_data, comparison_channel_names = convert_data_to_array(
    comparison_raw_data, file_extention

)

(
    comparison_base_data_df,
    comparison_filtered_data_rs,
    resampled_times,
) = prepare_comparison_data(comparison_data, config)
pr_freqs_rs, pr_fft_rs = prepare_fft(
    comparison_filtered_data_rs,
    config.FILTER_RANGE[0] - 5,
    config.FILTER_RANGE[1] + 5,
    config.BASE_SAMPLE_RATE,
)
# print columns of the dataframe
print(comparison_base_data_df.columns)

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(resampled_times, comparison_filtered_data_rs)
plt.title("Channel 1 - Channel 2  Filtered data")
plt.ylim(-200, 200)
# Plot the fft of the data
plt.figure(figsize=(15, 2))
plt.plot(
    resampled_times,
    (comparison_base_data_df[channel_1_key] - comparison_base_data_df[channel_2_key])
    * scale_factor,
)
plt.title("Channel 1 - Channel 2 unfiltered")

### Unpack IDUN data

In [ ]:
idun_base_data, idun_filtered_data, idun_time_stamps = prepare_idun_data(
    idun_raw_data, config
)

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(idun_time_stamps, idun_filtered_data)
plt.ylim(-200, 200)
plt.title("IDUN EMG filtered data")

plt.figure(figsize=(15, 2))
plt.plot(idun_time_stamps, idun_base_data)
plt.ylim(-200, 200)
plt.title("IDUN Highpassed data")

### Plot together

In [ ]:
# Create a figure and axis
plt.figure(figsize=(15, 2))  # Adjust the figsize as needed
ax = plt.gca()
plt.plot(
    resampled_times,
    comparison_filtered_data_rs,
    label="Channel 1 - Channel 2 Filtered Data",
)
plt.plot(idun_time_stamps, idun_filtered_data, label="IDUN EMG Filtered Data")
plt.ylim(-200, 200)
plt.legend()
plt.title("Superimposed Filtered Data")
plt.show()

### Make data same length and do auto shifting based on time

In [ ]:
# To use the function
(
    comparison_clipped_data,
    idun_clipped_data,
    idun_base_clipped_data,
    comparison_base_clipped_df,
    same_times,
) = sync_start_and_equalize_data_length(
    comparison_filtered_data_rs,
    idun_filtered_data,
    idun_base_data,
    comparison_base_data_df,
    comparison_time_stamps,
    idun_time_stamps,
    config,
)

## Do extra shift if necessary

In [ ]:
comparison_clipped_data_manual, idun_clipped_data_manual, idun_base_clipped_data_manual, comparison_base_clipped_df_manual, same_times = manual_sync(comparison_clipped_data, idun_clipped_data, idun_base_clipped_data, comparison_base_clipped_df, config)

In [ ]:
x_start = 0
range_viz = 25000
# create indeces from second limits
x_start_index = int(x_start * config.BASE_SAMPLE_RATE)
range_viz_index = int(range_viz * config.BASE_SAMPLE_RATE)


plt.figure(figsize=(15, 2))
plt.plot(
    same_times[x_start_index : x_start_index + range_viz_index],
    idun_clipped_data_manual[x_start_index : x_start_index + range_viz_index],
    label="idun",
)
plt.plot(
    same_times[x_start_index : x_start_index + range_viz_index],
    0.5 * comparison_clipped_data_manual[x_start_index : x_start_index + range_viz_index],
    label=config.DEVICE,
)
plt.legend("lower right")
plt.ylim(-100, 100)
plt.grid()

### Calculate lag first iteration

In [ ]:
# extract only the first 1000 seconds
comparison_clipped_temp_data = copy.deepcopy(comparison_clipped_data_manual)
idun_clipped_temp_data = copy.deepcopy(idun_clipped_data_manual)

search_size = config.FIRST_LAG_EPOCH_SIZE

comparison_epochs = epoch_data(comparison_clipped_temp_data, search_size)
idun_epochs = epoch_data(idun_clipped_temp_data, search_size)
# extract only the f

In [ ]:
correlation_arr, max_corr_arr, lag_arr = calculate_epochs_lag(
    comparison_epochs, idun_epochs
)
lag_arr_copy = lag_arr[1:].copy()

In [ ]:
plt.figure(figsize=(15, 2))
# create a time array based on config.FIRST_LAG_EPOCH_SIZE and length of lag_arr_copy
plot_time_arr = np.linspace(
    0, len(lag_arr_copy) * config.FIRST_LAG_EPOCH_SIZE, len(lag_arr_copy)
)
# convert  to seconds
plot_time_arr = plot_time_arr / config.BASE_SAMPLE_RATE
plt.title("Lag over time")
plt.xlabel("Time (s)")
plt.ylabel("Lag (s)")
plt.plot(plot_time_arr, np.array(lag_arr_copy) / config.BASE_SAMPLE_RATE)

### Cut based on first analysis

In [ ]:
lag_arr = np.array(lag_arr)
lag_initial = lag_arr[:20]
lag_initial = remove_outliers(lag_initial)
lag_mean = lag_initial[0]

# cut the lag_mean data from the start of idun_clipped_data if it is positive or from the start of  if negative
if lag_mean < 0:
    idun_cut_data = idun_clipped_data_manual[-lag_mean:]
    idun_base_cut_data = idun_base_clipped_data_manual[-lag_mean:]
    comparison_cut_data = comparison_clipped_data_manual[:-(-lag_mean)]
    comparison_base_cut_df = comparison_base_clipped_df_manual[:-(-lag_mean)].reset_index(
        drop=True
    )
else:
    idun_cut_data = idun_clipped_data_manual[:-(lag_mean)]
    idun_base_cut_data = idun_base_clipped_data_manual[:-(lag_mean)]
    comparison_cut_data = comparison_clipped_data_manual[lag_mean:]
    comparison_base_cut_df = comparison_base_clipped_df_manual[lag_mean:].reset_index(
        drop=True
    )

In [ ]:
# plot the data
plt.figure(figsize=(15, 2))
plt.plot(lag_initial / config.BASE_SAMPLE_RATE, label="idun")
print("lag_mean: ", lag_mean / config.BASE_SAMPLE_RATE)

In [ ]:
# plot the data
plt.figure(figsize=(15, 2))
time_seconds = np.arange(len(idun_cut_data)) / config.BASE_SAMPLE_RATE
plt.plot(
    time_seconds[x_start_index : x_start_index + range_viz_index],
    idun_cut_data[x_start_index : x_start_index + range_viz_index],
    label="idun",
)
plt.plot(
    time_seconds[x_start_index : x_start_index + range_viz_index],
    0.5 * comparison_cut_data[x_start_index : x_start_index + range_viz_index],
    label=config.DEVICE,
)
plt.legend("lower right")
plt.ylim(-100, 100)
plt.grid()

### Do a more fine grained sync

In [ ]:
cutted_comparisoneeg_epochs = epoch_data(
    comparison_cut_data, config.SECOND_LAG_EPOCH_SIZE
)
cutted_idun_epochs = epoch_data(idun_cut_data, config.SECOND_LAG_EPOCH_SIZE)

In [ ]:
fine_correlation_arr, fine_max_corr_arr, fine_lag_arr = calculate_epochs_lag(
    cutted_comparisoneeg_epochs, cutted_idun_epochs
)
fine_lag_arr[0] = fine_lag_arr[1]  # the very first lag estimation is not accurate

In [ ]:
fine_lag_arr_copy = fine_lag_arr.copy()
plt.figure(figsize=(15, 2))
plt.plot(np.array(fine_lag_arr_copy) / 250)
plt.title("Lag")
plt.xlabel(f"Epochs of {config.SECOND_LAG_EPOCH_SIZE} samples")
plt.ylabel("Lag (s)")
plt.ylim(-15, 15)

### Do some cleaning on lag estimation

### ------------------------------------------------------------------------------------------------

In [ ]:
DISCONTINUITY_THRESHOLD = config.DISCONTINUITY_THRESHOLD
# save as txt file the correlation strictness
np.savetxt(
    os.path.join(folder, subject, night, "discontinuity_threshold.txt"),
    [DISCONTINUITY_THRESHOLD],
    delimiter=",",
)

### ------------------------------------------------------------------------------------------------

In [ ]:
cleaned_fine_lag_arr = clean_data_from_spikes(
    fine_lag_arr_copy, DISCONTINUITY_THRESHOLD
)
plt.figure(figsize=(15, 2))
plt.plot(cleaned_fine_lag_arr / config.BASE_SAMPLE_RATE)
plt.title("Lag")

### Make signal whole

In [ ]:
# interpolate the signal
interpolated_cleaned_fine_lag_arr = interpolate_signal(cleaned_fine_lag_arr)
smoothed_lag_arr = smooth(interpolated_cleaned_fine_lag_arr, window_size=10)

In [ ]:
# Plotting the regression curve
plt.figure(figsize=(15, 2))
plt.plot(
    cleaned_fine_lag_arr / config.BASE_SAMPLE_RATE,
    label="Actual Lag estimation",
    alpha=0.5,
)
plt.plot(
    smoothed_lag_arr / config.BASE_SAMPLE_RATE,
    "r-",
    label="Final lag estimation",
    linewidth=1,
)
plt.title("Estimation of lag between two devices")
plt.xlabel("Time in seconds")
plt.ylabel("Lag in seconds")
plt.legend()
plt.savefig(
    os.path.join(folder, subject, night, "lag_estimation.png"),
    dpi=300,
    bbox_inches="tight",
)

### Create linear knots

In [ ]:
x_axis_simulated = np.linspace(
    0, len(smoothed_lag_arr) - 1, len(smoothed_lag_arr)
)  # This will be 0 to 718

x_knots = np.linspace(0, len(smoothed_lag_arr) - 1, config.TOTAL_LINEAR_SEGMENTS + 1)
y_knots = smoothed_lag_arr[
    (x_knots).astype(int)
]  # Directly get the y-values from y_pred

In [ ]:
# Plot
plt.figure(figsize=(15, 2))
plt.plot(x_axis_simulated, smoothed_lag_arr, label="Original Curve")
plt.plot(
    x_knots,
    y_knots,
    label=f"{config.TOTAL_LINEAR_SEGMENTS} Linear Segments",
    linestyle="--",
    marker="o",
    linewidth=1,
    markersize=1,
)
plt.legend()
plt.show()

### Use the Linear lines to estimate the difference in sampling rates and apply

In [ ]:
point_size = config.SECOND_LAG_EPOCH_SIZE
adjustment_epoch_size = int(len(comparison_cut_data) / config.TOTAL_LINEAR_SEGMENTS)
lag_positions = np.arange(0, len(comparison_cut_data), adjustment_epoch_size)
cumulative_lags = y_knots.astype(int)
lag_sizes = np.diff(cumulative_lags)
# make sure lag_sizes and lag_positions have the same length by cutting end of shorter one
if len(lag_sizes) > len(lag_positions):
    lag_sizes = lag_sizes[: len(lag_positions)]
elif len(lag_sizes) < len(lag_positions):
    lag_positions = lag_positions[: len(lag_sizes)]

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(lag_sizes)
plt.title("Number of samples to be removed at each lag")
plt.axhline(y=0, color="r", linestyle="--")
plt.ylim(-10, 10)

### Cut the data throughout and at the end

In [ ]:
(
    idun_data_pruned,
    idun_base_pruned,
    comparison_data_pruned,
    comparison_base_pruned_df,
) = cut_throughout_data_dual(
    idun_cut_data,
    idun_base_cut_data,
    comparison_cut_data,
    comparison_base_cut_df,
    lag_positions,
    lag_sizes,
)

In [ ]:
final_comparison_epochs = epoch_data(
    comparison_data_pruned, config.SECOND_LAG_EPOCH_SIZE
)
final_idun_epochs = epoch_data(idun_data_pruned, config.SECOND_LAG_EPOCH_SIZE)

In [ ]:
final_correlation_arr, final_max_corr_arr, final_lag_arr = calculate_epochs_lag(
    final_comparison_epochs, final_idun_epochs
)

In [ ]:
final_lag_arr_copy = final_lag_arr.copy()
plt.figure(figsize=(15, 2))
plt.plot(np.array(final_lag_arr) / config.BASE_SAMPLE_RATE)
plt.title("Lag")
plt.ylim(-0.3, 0.3)
plt.grid()

### Clean the lags

In [ ]:
cleaned_final_lag_arr = replace_outliers(final_lag_arr_copy)
# rfind where the values are nan
nan_idx = np.argwhere(np.isnan(cleaned_final_lag_arr))
# remove the nan values
mean_final_lag = np.mean(np.delete(cleaned_final_lag_arr, nan_idx, axis=0))

In [ ]:
plt.figure(figsize=(15, 2))
plt.plot(np.array(cleaned_final_lag_arr) / config.BASE_SAMPLE_RATE)
# plot the mean_final_lag as a horizontal line
plt.axhline(y=mean_final_lag / config.BASE_SAMPLE_RATE, color="r", linestyle="--")
plt.title("Lag")
plt.ylim(-1, 1)
plt.grid()

### Remove this final shift

In [ ]:
(
    shifted_final_comparisoneeg_arr,
    shifted_final_comparisoneeg_base_df,
    shifted_final_idun_arr,
    shifted_final_idun_base_arr,
) = adjust_data_by_mean_lag(
    mean_final_lag,
    comparison_data_pruned,
    comparison_base_pruned_df,
    idun_data_pruned,
    idun_base_pruned,
)

In [ ]:
shifted_final_comparisoneeg_epochs = epoch_data(
    shifted_final_comparisoneeg_arr, config.SECOND_LAG_EPOCH_SIZE
)
shifted_final_idun_epochs = epoch_data(
    shifted_final_idun_arr, config.SECOND_LAG_EPOCH_SIZE
)
(
    shifted_final_correlation_arr,
    shifted_final_max_corr_arr,
    shifted_final_lag_arr,
) = calculate_epochs_lag(shifted_final_comparisoneeg_epochs, shifted_final_idun_epochs)

In [ ]:
plt.figure(figsize=(15, 2))
# create time array kby muliplying the length of the array by the epoch size
time_seconds = (
    np.arange(len(shifted_final_lag_arr))
    * config.SECOND_LAG_EPOCH_SIZE
    / config.BASE_SAMPLE_RATE
)
plt.plot(time_seconds, np.array(shifted_final_lag_arr) / config.BASE_SAMPLE_RATE)
plt.axhline(y=0, color="r", linestyle="--")
plt.title("Lag")
plt.xlabel("Time in seconds")
plt.ylabel("Lag in seconds")
plt.ylim(-0.05, 0.05)
plt.grid()
# save image of lag
plt.savefig(os.path.join(folder, subject, night, f"{subject}_{night}_lag.png"))

### Plot final filtered data

In [ ]:
plt.figure(figsize=(15, 2))
# create a seconds axis
time_seconds = np.arange(len(shifted_final_idun_arr)) / config.BASE_SAMPLE_RATE
plt.plot(shifted_final_idun_arr, label="idun")
plt.plot(0.5 * shifted_final_comparisoneeg_arr, label=config.DEVICE)
plt.legend()
plt.ylim(-100, 100)
plt.grid()

### Final validation, plot final raw data

In [ ]:
print(shifted_final_comparisoneeg_base_df.shape)
print(shifted_final_idun_base_arr.shape)

comparisoneeg_channel_1_data = np.array(
    shifted_final_comparisoneeg_base_df[channel_1_key]
)
comparisoneeg_channel_2_data = np.array(
    shifted_final_comparisoneeg_base_df[channel_2_key]
)
# minus right eye from left eye
comparisoneeg_channel_1_minus_2 = (
    comparisoneeg_channel_1_data - comparisoneeg_channel_2_data
)
comparisoneeg_channel_1_minus_2 = (
    comparisoneeg_channel_1_minus_2 * scale_factor
)  # To get the data to same scale as ours, v to uv

comparisoneeg_filtered = do_bandpass(
    comparisoneeg_channel_1_minus_2,
    [config.FILTER_RANGE[0], config.FILTER_RANGE[1]],
    config.BASE_SAMPLE_RATE,
)

idun_filtered = do_bandpass(
    shifted_final_idun_base_arr,
    [config.FILTER_RANGE[0], config.FILTER_RANGE[1]],
    config.BASE_SAMPLE_RATE,
)


plt.figure(figsize=(15, 2))
time_seconds = np.arange(len(shifted_final_idun_base_arr)) / config.BASE_SAMPLE_RATE
plt.plot(time_seconds, shifted_final_idun_base_arr, label="idun")
plt.plot(time_seconds, 0.5 * comparisoneeg_channel_1_minus_2, label=config.DEVICE)
plt.legend()
plt.title("Raw and synced data")
plt.xlabel("Time in seconds")
plt.ylabel("Amplitude in uV")
plt.ylim(-100, 100)
plt.savefig(os.path.join(folder, subject, night, f"{subject}_{night}_raw.png"))
plt.grid()


plt.figure(figsize=(15, 2))
time_seconds = np.arange(len(idun_filtered)) / config.BASE_SAMPLE_RATE
plt.plot(time_seconds, idun_filtered, label="idun")
plt.plot(time_seconds, 0.5 * comparisoneeg_filtered, label=config.DEVICE)
plt.legend()
plt.ylim(-100, 100)
plt.title("Filtered and synced data")
plt.xlabel("Time in seconds")
plt.ylabel("Amplitude in uV")
plt.savefig(os.path.join(folder, subject, night, f"{subject}_{night}_filtered.png"))
plt.grid()

In [ ]:
plt.figure(figsize=(15, 2))
time_seconds = np.arange(len(idun_filtered)) / config.BASE_SAMPLE_RATE
plt.plot(
    time_seconds[x_start_index : x_start_index + range_viz_index],
    idun_filtered[x_start_index : x_start_index + range_viz_index],
    label="idun",
)
plt.plot(
    time_seconds[x_start_index : x_start_index + range_viz_index],
    0.2 * comparisoneeg_filtered[x_start_index : x_start_index + range_viz_index],
    label=config.DEVICE,
)
plt.legend("lower right")
plt.ylim(-25, 25)
plt.title("Filtered data")
plt.xlabel("Time in seconds")
plt.ylabel("Amplitude in uV")
plt.grid()
plt.savefig(
    os.path.join(folder, subject, night, f"{subject}_{night}_filtered_zoom.png")
)

### Combine Dataframes

In [ ]:
# create a time array based on config.BASE_SAMPLE_RATE and length of prodigy_filtered
time_array = np.linspace(
    0,
    len(shifted_final_idun_base_arr) / config.BASE_SAMPLE_RATE,
    len(shifted_final_idun_base_arr),
)
# copy the shifted_final_prodigy_base_df
synced_base_df_copy = copy.deepcopy(shifted_final_comparisoneeg_base_df)
# add shifted_final_idun_base_arr as "IDUN" column to prodigy_base_df_copy
synced_base_df_copy["IDUN"] = shifted_final_idun_base_arr
# make index of prodigy_base_df_copy as time_array
synced_base_df_copy.index = time_array

# save the dataframe as csv, and add a header indicating the sampling rate
save_path = os.path.join(folder, subject, night, f"{subject}_{night}_synced_data.csv")
synced_base_df_copy.to_csv(save_path, index=True)